In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
URL = 'https://www.indeed.com/jobs?q=data%20scientist&l=New%20York&vjk=d3fe193eb8430aa2'

#conducting a request of the stated URL above:
page = requests.get(URL)

#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, 'html.parser')

#printing soup in a more structured tree format that makes for easier reading
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script src="//d3fw5vlhllyvee.cloudfront.net/s/1d80eda/en_US.js" type="text/javascript">
  </script>
  <link href="//d3fw5vlhllyvee.cloudfront.net/s/b45d10b/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="https://rss.indeed.com/rss?q=data+scientist&amp;l=New+York" rel="alternate" title="Data Scientist Jobs, Employment in New York State" type="application/rss+xml"/>
  <link href="/m/jobs?q=data+scientist&amp;l=New+York" media="only screen and (max-width: 640px)" rel="alternate"/>
  <link href="/m/jobs?q=data+scientist&amp;l=New+York" media="handheld" rel="alternate"/>
  <script type="text/javascript">
   if (typeof window['closureReadyCallbacks'] == 'undefined') {
window['closureReadyCallbacks'] = [];
}

function call_when_jsall_loaded(cb) {
if (window['closureReady']) {
cb();
} else {
window['closureReadyCallbacks'].push(cb);
}
}
  </script>
  <

In [21]:
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}): jobs.append(a["title"])
    return(jobs)
extract_job_title_from_result(soup)

['Data Scientist',
 'Data Scientist',
 'Senior Data Scientist (Remote-friendly)',
 'Machine Learning Engineer',
 'Data Scientist, Messaging',
 'Data Scientist',
 'Data Scientist',
 'Data Scientist',
 'Senior Data Scientist',
 'Data Scientist',
 'eCom Senior Data Scientist',
 'Data Scientist',
 'Product Data Scientist',
 'Data Scientist',
 'Senior Data Scientist - Tech Incubation']

In [24]:
def extract_company_from_result(soup): 
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)
 
extract_company_from_result(soup)

['Triplebyte',
 'Building Service 32BJ Benefit Funds',
 'Noom Inc.',
 'MORI Associates, Inc.',
 'Spotify',
 'Publishers Clearing House',
 'Beauty Matching Engine',
 'CVS Health',
 'CVS Health',
 'Source Enterprises',
 'PepsiCo',
 'Roivant Sciences',
 'Twitter',
 'CDPHP',
 'Memorial Sloan-Kettering Cancer Center']

In [25]:
def extract_location_from_result(soup): 
    locations = []
    spans = soup.findAll("span", attrs={"class": "location"})
    for span in spans:
        locations.append(span.text)
    return(locations)

extract_location_from_result(soup)

['New York, NY 10011 (Flatiron District area)',
 'Jericho, NY 11753',
 'New York, NY',
 'New York, NY',
 'New York, NY',
 'New York, NY',
 'New York, NY 10017 (Murray Hill area)',
 'New York State',
 'New York, NY',
 'Albany, NY 12206 (Campus Area (University District) area)']

In [30]:
def extract_salary_from_result(soup): 
    salaries = []
    for div in soup.find_all(name="span", attrs={"class":"row"}):
        try:
            salaries.append(span.find("salaryText").text)
        except:
            salaries.append("Nothing_found")
    return(salaries)

extract_salary_from_result(soup)

['Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found']

In [34]:
def extract_summary_from_result(soup): 
    summaries = []
    divs = soup.findAll("div", attrs={"class": "summary"})
    for li in divs:
        summaries.append(li.text.strip())
    return(summaries)

extract_summary_from_result(soup)

["You’ll report directly to Triplebytes' Head of Machine Learning and will work alongside a team of 6-8 machine learning engineers and data scientists.",
 '\uf0b7 A proven passion for generating insights from data.\n\uf0b7 Present information using data visualization techniques.\n\uf0b7 Data mining using state-of-the-art methods.',
 'You are comfortable with at least “medium data” technologies and how to transcend the “memory bound” nature of most analytics tools.',
 'You have previous experience working as an engineer or data scientist.\nYou understand data structures, data modeling, and the basics of.',
 'Mentor and coach other data scientists.\nYou have the technical competence to perform advanced analytics: coding skills (such as Python, Java, or Scala),…',
 'Utilize large-scale data mining and machine learning techniques to discover actionable patterns in the data.\nWork comfortably with multi-terabyte, and billion+…',
 'MS or PhD in computer science, data mining, machine learning

In [36]:
max_results_per_city = 800
city_set = ['New+York','Chicago','San+Francisco', 'Austin', 'Seattle', 'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 
            'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Washington+DC', 'Boulder']
columns = ["city", "job_title", "company_name", "location", "summary", "salary"]
sample_df = pd.DataFrame(columns = columns)
sample_df

Empty DataFrame
Columns: [city, job_title, company_name, location, summary, salary]
Index: []

In [41]:
for city in city_set:
    for start in range(0, max_results_per_city, 10):
        page = requests.get('http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=' + str(city) + '&start=' + str(start))
        time.sleep(1)  #ensuring at least 1 second between page grabs
        soup = BeautifulSoup(page.text, "lxml", from_encoding="utf-8")
    for div in soup.find_all(name="div", attrs={"class":"row"}): 
    #specifying row num for index of job posting in dataframe
        num = (len(sample_df) + 1) 
    #creating an empty list to hold the data for each posting
        job_post = [] 
    #append city name
        job_post.append(city) 
    #grabbing job title
    for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
        job_post.append(a["title"]) 
    #grabbing company name
        company = div.find_all(name="span", attrs={"class":"company"}) 
    if len(company) > 0: 
        for b in company:
            job_post.append(b.text.strip()) 
    else: 
        sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
    for span in sec_try:
        job_post.append(span.text) 
    #grabbing location name
    c = div.findAll('span', attrs={'class': 'location'}) 
    for span in c: 
        job_post.append(span.text) 
    #grabbing summary text
    d = div.findAll('span', attrs={'class': 'summary'}) 
    for span in d:
        job_post.append(span.text.strip()) 
    #grabbing salary
    try:
        job_post.append(div.find('salaryText').text) 
    except:
        job_post.append("Nothing_found") 
    #appending list of job post info to dataframe at index num
    sample_df.loc[num] = job_post

#saving sample_df as a local csv file — define your own local path to save contents 
sample_df.to_csv("[filepath].csv", encoding='utf-8')

extract_salary_from_result(soup)

C:\Users\megan.moroney\Anaconda3\lib\site-packages\bs4\__init__.py:203: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


NameError: name 'sec_try' is not defined